# Preprocessing the XLSX Data

## TODO: Add Stations related info to the measurement files, preprocess the data for NN and Time Series models (you can safely start from air_quality_2019_2023.csv)

## Imports

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path


## Stations
- renaming columns

In [13]:
# Read the Excel file (specific sheet)
df = pd.read_excel("../data/raw/Stations.xlsx")

df = df.rename(columns={
    "Station Code" : "station_code",
    "International Code": "int_code",
    "Street address": "address",
    "Area Type": "area_type",
    "Station Category": "station_category",
    "WGS84 φ N": "latitude",
    "WGS84 λ E": "longitude"
})

print(df.head())

df.to_csv("../data/processed/stations.csv", index=False)

   station_code int_code                     address area_type  \
0  MpKrakAlKras  PL0012A  Kraków, Aleja Krasińskiego     urban   
1  MpKrakBujaka  PL0501A          Kraków, ul. Bujaka     urban   
2  MpKrakBulwar  PL0039A       Kraków, ul. Bulwarowa     urban   
3  MpKrakOsPias  PL0642A         Kraków, os. Piastów     urban   
4  MpKrakSwoszo  PL0735A      Kraków, os. Swoszowice     urban   

             station_category   latitude  longitude  
0   stationary container unit  50.057678  19.926189  
1   stationary container unit  50.010575  19.949189  
2   stationary container unit  50.069308  20.053492  
3  free-standing dust sampler  50.098508  20.018269  
4  free-standing dust sampler  49.991442  19.936792  


## Measurements Merge
- merging measurements from 2019 to 2023
- cleaning datetime (2020 had another format of time (07:59:59.712000)

In [25]:
data_dir = Path("../data/raw/AirQuality_Krakow")
files = sorted(data_dir.glob("*.xlsx"))

dfs = []
for f in files:
    print(f"\n=== Processing file: {f.name} ===")
    df = pd.read_excel(f, dtype=str)

    # keep raw for cleaning only
    df["DateTime_clean"] = df["DateTime"].str.split(".").str[0]
    df["DateTime"] = pd.to_datetime(df["DateTime_clean"], errors="coerce")

    # sanity check
    if df["DateTime"].isna().any():
        print(f"⚠️ Found {df['DateTime'].isna().sum()} bad rows in {f.name}")
    else:
        print("✅ All DateTime parsed correctly.")

    # drop helpers
    df = df.drop(columns=["DateTime_clean"])
    dfs.append(df)

# Merge into one DataFrame
data = pd.concat(dfs, ignore_index=True).sort_values("DateTime")

print("\nFinal merged shape:", data.shape)
print(data.head())

# Save to CSV
data.to_csv("../data/processed/air_quality_2019_2023.csv", index=False)
print("✅ Saved clean dataset to ../data/processed/air_quality_2019_2023.csv")


=== Processing file: 2019_PM10_1g.xlsx ===
✅ All DateTime parsed correctly.

=== Processing file: 2020_PM10_1g.xlsx ===
✅ All DateTime parsed correctly.

=== Processing file: 2021_PM10_1g.xlsx ===
✅ All DateTime parsed correctly.

=== Processing file: 2022_PM10_1g.xlsx ===
✅ All DateTime parsed correctly.

=== Processing file: 2023_PM10_1g.xlsx ===
✅ All DateTime parsed correctly.

Final merged shape: (43824, 8)
             DateTime MpKrakAlKras MpKrakBujaka MpKrakBulwar MpKrakOsPias  \
0 2019-01-01 01:00:00      88.2185       139.79      120.057      161.053   
1 2019-01-01 02:00:00      95.2209      92.5099      63.4217      62.3289   
2 2019-01-01 03:00:00      85.3689      57.1358      48.6426      56.5202   
3 2019-01-01 04:00:00      70.8575      39.4615      36.7828      49.5539   
4 2019-01-01 05:00:00      50.1935      27.1423      28.7538      36.1824   

  MpKrakSwoszo MpKrakWadow MpKrakZloRog  
0          NaN     83.6841      77.8355  
1          NaN     66.3402      82.9

## Date Features Engineering for Tree-Based Models
- extracting date related features for tree-based models
- cyclic encoding of hour of the day, day of year and weekday
- dropping of the original dateTime column

In [27]:
# Load your merged CSV
data = pd.read_csv("../data/processed/air_quality_2019_2023.csv", parse_dates=["DateTime"])

# Extract basic time features
data["year"] = data["DateTime"].dt.year
data["month"] = data["DateTime"].dt.month
data["day"] = data["DateTime"].dt.day
data["weekday"] = data["DateTime"].dt.weekday  # Monday=0
data["hour"] = data["DateTime"].dt.hour
data["day_of_year"] = data["DateTime"].dt.dayofyear

# Cyclic encoding for hour (24h cycle)
data["hour_sin"] = np.sin(2 * np.pi * data["hour"] / 24)
data["hour_cos"] = np.cos(2 * np.pi * data["hour"] / 24)

# Cyclic encoding for day of year (seasonality)
data["doy_sin"] = np.sin(2 * np.pi * data["day_of_year"] / 365.25)
data["doy_cos"] = np.cos(2 * np.pi * data["day_of_year"] / 365.25)

# Optional: encode weekday cyclically
data["weekday_sin"] = np.sin(2 * np.pi * data["weekday"] / 7)
data["weekday_cos"] = np.cos(2 * np.pi * data["weekday"] / 7)

# Drop the original DateTime if you want only numeric features for trees
data = data.drop(columns=["DateTime"])

# Save the dataset ready for tree-based ML
data.to_csv("../data/processed/air_quality_2019_2023_tree_features.csv", index=False)
print("✅ Saved dataset with tree-based features")
print(data.head())

✅ Saved dataset with tree-based features
   MpKrakAlKras  MpKrakBujaka  MpKrakBulwar  MpKrakOsPias  MpKrakSwoszo  \
0       88.2185      139.7900      120.0570      161.0530           NaN   
1       95.2209       92.5099       63.4217       62.3289           NaN   
2       85.3689       57.1358       48.6426       56.5202           NaN   
3       70.8575       39.4615       36.7828       49.5539           NaN   
4       50.1935       27.1423       28.7538       36.1824           NaN   

   MpKrakWadow  MpKrakZloRog  year  month  day  weekday  hour  day_of_year  \
0      83.6841       77.8355  2019      1    1        1     1            1   
1      66.3402       82.9678  2019      1    1        1     2            1   
2      55.8833       64.5276  2019      1    1        1     3            1   
3      44.1614       46.5672  2019      1    1        1     4            1   
4      34.5853       48.1257  2019      1    1        1     5            1   

   hour_sin  hour_cos   doy_sin   doy_c

## Melting the data - 1 row per measurement per station
- breaking the original file from wide (1 row per timestamp) to long (1 row per timestamp per station)
- dropping of N/A rows for training purposes

In [2]:
import pandas as pd

# Load your cleaned dataset with tree features
data = pd.read_csv("../data/processed/air_quality_2019_2023_tree_features.csv")

# Identify the station columns
station_cols = [
    "MpKrakAlKras", "MpKrakBujaka", "MpKrakBulwar", 
    "MpKrakOsPias", "MpKrakSwoszo", "MpKrakWadow", "MpKrakZloRog"
]

# Melt to long format
data_long = data.melt(
    id_vars=[
        "year", "month", "day", "weekday", "hour", "day_of_year",
        "hour_sin", "hour_cos", "doy_sin", "doy_cos", "weekday_sin", "weekday_cos"
    ],
    value_vars=station_cols,
    var_name="station_code",
    value_name="pm10"
)

# Sort by station and datetime for proper lag computation
data_long = data_long.sort_values(["station_code", "year", "month", "day", "hour"])

# Compute lag features per station
lag_hours = [1, 2]  # lag 1 hour, 2 hours
for lag in lag_hours:
    data_long[f"pm10_lag_{lag}"] = data_long.groupby("station_code")["pm10"].shift(lag)

# Drop rows where pm10 is missing
data_long = data_long.dropna(subset=["pm10"])

# Optional: reset index
data_long = data_long.reset_index(drop=True)

print("Long format shape:", data_long.shape)
print(data_long.head())

# Save the long format dataset ready for general model
data_long.to_csv("../data/processed/air_quality_2019_2023_long.csv", index=False)
print("✅ Saved long-format dataset with PM10 lag features.")

Long format shape: (290736, 16)
   year  month  day  weekday  hour  day_of_year  hour_sin  hour_cos   doy_sin  \
0  2019      1    1        1     1            1  0.258819  0.965926  0.017202   
1  2019      1    1        1     2            1  0.500000  0.866025  0.017202   
2  2019      1    1        1     3            1  0.707107  0.707107  0.017202   
3  2019      1    1        1     4            1  0.866025  0.500000  0.017202   
4  2019      1    1        1     5            1  0.965926  0.258819  0.017202   

    doy_cos  weekday_sin  weekday_cos  station_code     pm10  pm10_lag_1  \
0  0.999852     0.781831      0.62349  MpKrakAlKras  88.2185         NaN   
1  0.999852     0.781831      0.62349  MpKrakAlKras  95.2209     88.2185   
2  0.999852     0.781831      0.62349  MpKrakAlKras  85.3689     95.2209   
3  0.999852     0.781831      0.62349  MpKrakAlKras  70.8575     85.3689   
4  0.999852     0.781831      0.62349  MpKrakAlKras  50.1935     70.8575   

   pm10_lag_2  
0       

In [33]:
print(data_long.shape)

(290736, 14)


In [3]:
data_tree = pd.read_csv("../data/processed/air_quality_2019_2023_tree_features.csv")
print(data_tree.shape)

(43824, 19)
